## Import

In [44]:
import sqlite3
import pandas as pd
import datetime

from sqlalchemy import create_engine
import urllib
import pyodbc

### Load data

In [116]:
data=pd.read_csv('D:\junior__test_DS\sql\shedulers.csv', encoding='cp1251', sep=';')
data

,ФИО,Расписание,Дата начала расписания,Дата окончания расписания
0,Поставщик 1,дддвсвнн,01.01.2019 0:00,10.01.2019 0:00
1,Поставщик 1,ннвннв,11.01.2019 0:00,15.01.2019 0:00
2,Поставщик 1,св,16.01.2019 0:00,20.01.2019 0:00
3,Поставщик 2,свсвсв,01.01.2019 0:00,07.01.2019 0:00
4,Поставщик 2,днвсв,08.01.2019 0:00,14.01.2019 0:00
5,Поставщик 2,ннддвсв,15.01.2019 0:00,31.12.9999 0:00
6,Поставщик 3,нвнвнв,01.01.2019 0:00,01.02.2019 0:00
7,Поставщик 3,двдвдвдв,02.02.2019 0:00,31.12.9999 0:00


**P.S. Некоторые примеры в задании не соответствуют предоставленным данным. Вероятно, данные изменили (поле расписание), а формулировка задания осталась - поэтому такой gap.**

## Задание 1. Создать таблицу T_CONTRACTOR_SHEDULER и загрузить в неё данные

In [46]:
quoted = urllib.parse.quote_plus("DRIVER={SQL Server};SERVER=DESKTOP-OM56TJJ\SQLEXPRESS;DATABASE=testdb")
engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))

In [48]:
engine.execute('CREATE TABLE T_CONTRACTOR_SHEDULER (id INT, name VARCHAR(20),shedule VARCHAR(10),date_begin DATE,date_end DATE, PRIMARY KEY(id))')

In [49]:
# переименуем колонки для удобства
data_load = data.reset_index()
data_load.columns=['id','name', 'shedule', 'date_begin', 'date_end']

In [50]:
data_load.to_sql('T_CONTRACTOR_SHEDULER3', schema='dbo', con = engine, chunksize=200, method='multi', index=False, if_exists='replace')

In [51]:
result = engine.execute('SELECT * FROM T_CONTRACTOR_SHEDULER3')
result.fetchall()

[(0, 'Поставщик 1', 'дддвсвнн', '01.01.2019 0:00', '10.01.2019 0:00'),
 (1, 'Поставщик 1', 'ннвннв', '11.01.2019 0:00', '15.01.2019 0:00'),
 (2, 'Поставщик 1', 'св', '16.01.2019 0:00', '20.01.2019 0:00'),
 (3, 'Поставщик 2', 'свсвсв', '01.01.2019 0:00', '07.01.2019 0:00'),
 (4, 'Поставщик 2', 'днвсв', '08.01.2019 0:00', '14.01.2019 0:00'),
 (5, 'Поставщик 2', 'ннддвсв', '15.01.2019 0:00', '31.12.9999 0:00'),
 (6, 'Поставщик 3', 'нвнвнв', '01.01.2019 0:00', '01.02.2019 0:00'),
 (7, 'Поставщик 3', 'двдвдвдв', '02.02.2019 0:00', '31.12.9999 0:00')]

## Задание 2. Создать таблицу T_CONTRACTOR_WORK_DAY

<code>CREATE TABLE T_CONTRACTOR_WORK_DAY(
ID INT PRIMARY KEY,
name VARCHAR(15),
date_begin DATE,
date_end DATE)</code>

## Задание 3. Создать процедуру расчёта рабочих дней.

Честно, я не осилил реализовать работающую процедуру на своём сервере, бился долго. На Python(ниже), логика работает. Но на MSSQL ошибки и нездоровое поведение :\

Пришлось применить чит-код. Я загрузил таблицу со сгенёрнным ниже на python полным расписанием всех поставщиков в базу MSSQL, и написал процедуру, которая берёт данные уже из этой таблицы. Немного не красиво, но работает :)

In [131]:
total_schedule.to_sql('TOTAL_SCHEDULE1', schema='dbo', con = engine, chunksize=200, method='multi', index=False, if_exists='replace')

`CREATE PROCEDURE Get_Rasp1 @start_date DATE, @end_date DATE
AS
	UPDATE T_CONTRACTOR_WORK_DAY
	SET name=schedule.name,
		date_begin=schedule.start_d,
		date_end=schedule.end_d
		FROM
	(SELECT name, start_d, end_d
	FROM dbo.TOTAL_SCHEDULE1
	WHERE start_d BETWEEN @start_date AND @end_date) as schedule`


## Задание 4. 

Задание решил запросами к таблице с общим расписанием на год вперёд.

* Сделать выборку содержащую сколько рабочих дней было у каждого поставщика

`SELECT name, COUNT(end_d)
FROM [testdb].[dbo].[TOTAL_SCHEDULE1]
WHERE CAST(start_d AS DATE) < '2019-01-31'
GROUP BY name`

* Сделать выборку поставщиков, у которыйх было больше 10 рабочих дней за январь 2019 года

`SELECT name, COUNT(end_d) AS jobs
FROM [testdb].[dbo].[TOTAL_SCHEDULE1]
WHERE CAST(start_d AS DATE) < '2019-01-31'
GROUP BY name
HAVING COUNT(end_d)>10`

* Сделать выборку поставщиков, кто работал 14, 15 и 16 января 2019 года

`SELECT name
FROM [testdb].[dbo].[TOTAL_SCHEDULE1]
WHERE CAST(start_d AS DATE) BETWEEN '2019-01-14' AND '2019-01-17'
GROUP BY name
HAVING COUNT(start_d)>=3`

## 5. Задача 3 на Python.

In [117]:
# переименуем колонки для удобства
data.columns=['name', 'shedule', 'start_date', 'end_date']

In [118]:
# приводим к формату datetime
data.loc[5]['end_date']='31.12.2020 0:00'
data.loc[7]['end_date']='31.12.2020 0:00'
data['start_date'] = pd.to_datetime(data['start_date'], format='%d.%m.%Y %H:%M')
data['end_date'] = pd.to_datetime(data['end_date'], format='%d.%m.%Y %H:%M', errors='coerce')

In [119]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   name        8 non-null      object        
 1   shedule     8 non-null      object        
 2   start_date  8 non-null      datetime64[ns]
 3   end_date    8 non-null      datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 384.0+ bytes


In [121]:
# функция расчёта расписания в заданном интервале
# принимает даты, расписание на даты и имя поставщика
# строит расписание по дням и возвращает датафрейм с ним

def build_sch(s_date, e_date, sched, name):
    start_date=pd.to_datetime(s_date)
    end_date=pd.to_datetime(e_date)
    res=pd.DataFrame(columns=['name','start_d', 'end_d'])
    sched=sched*500
    curr_date=start_date
    for i in range((end_date+datetime.timedelta(days=1)-curr_date).days):
        if sched[i]=='д':
            start=curr_date+datetime.timedelta(hours=8)
            end=start+datetime.timedelta(hours=12)
            res.loc[curr_date]=[name, start, end]
            curr_date+=datetime.timedelta(days=1)
        elif sched[i]=='н':
            start=curr_date+datetime.timedelta(hours=20)
            end=start+datetime.timedelta(hours=12)
            res.loc[curr_date]=[name, start, end]
            curr_date+=datetime.timedelta(days=1)
        elif sched[i]=='с':
            start=curr_date+datetime.timedelta(hours=8)
            end=start+datetime.timedelta(hours=24)
            res.loc[curr_date]=[name, start, end]
            curr_date+=datetime.timedelta(days=1)
        elif sched[i]=='в':
            curr_date+=datetime.timedelta(days=1)
    return res

In [122]:
# глобальное расписание на ближайший год
total_schedule=pd.DataFrame(columns=['name','start_d', 'end_d'])

In [123]:
# функция для строк датафрейма
# проходит по строке и строит расписание по дням, на которые действует это расписание, используя функцию build_sch
# расписание за период дополняет в глобальное расписание для каждого поставщика
def get_sched(row):
    global total_schedule
    sched = row['shedule']
    s_date = row['start_date']
    e_date = row['end_date']
    n_name=row['name']
    curr_sched = build_sch(s_date, e_date, sched, n_name)
    total_schedule=total_schedule.append(curr_sched)

In [124]:
# строим общее расписание, проходя по каждой строке исходных данных
data.apply(get_sched, axis=1)

0    None
1    None
2    None
3    None
4    None
5    None
6    None
7    None
dtype: object

In [125]:
# целевая функция, возвращающая расписание за любой указанный период
def get_result(s_date, e_date):
    data = total_schedule.query('@s_date<start_d<@e_date')
    data.reset_index(drop=True, inplace=True)
    return data

In [126]:
get_result('2019-01-01', '2019-01-08')

,name,start_d,end_d
0,Поставщик 1,2019-01-01 08:00:00,2019-01-01 20:00:00
1,Поставщик 1,2019-01-02 08:00:00,2019-01-02 20:00:00
2,Поставщик 1,2019-01-03 08:00:00,2019-01-03 20:00:00
3,Поставщик 1,2019-01-05 08:00:00,2019-01-06 08:00:00
4,Поставщик 1,2019-01-07 20:00:00,2019-01-08 08:00:00
5,Поставщик 2,2019-01-01 08:00:00,2019-01-02 08:00:00
6,Поставщик 2,2019-01-03 08:00:00,2019-01-04 08:00:00
7,Поставщик 2,2019-01-05 08:00:00,2019-01-06 08:00:00
8,Поставщик 2,2019-01-07 08:00:00,2019-01-08 08:00:00
9,Поставщик 3,2019-01-01 20:00:00,2019-01-02 08:00:00


# 6. Задача 4 с помощью Python:

* Сделать выборку содержащую сколько рабочих дней было у каждого поставщика

In [127]:
# тут не очень понятно за какой период спрашивается...
# ну допустим за январь 2019

jan_works = get_result('2019-01-01', '2019-01-31')
jan_works.groupby('name').count()

,start_d,end_d
name,,
Поставщик 1,15,15
Поставщик 2,21,21
Поставщик 3,15,15


* Сделать выборку поставщиков, у которыйх было больше 10 рабочих дней за январь 2019 года

In [129]:
jan_works.groupby('name').count()['start_d']>10

name
Поставщик 1    True
Поставщик 2    True
Поставщик 3    True
Name: start_d, dtype: bool

* Сделать выборку поставщиков, кто работал 14, 15 и 16 января 2019 года

In [130]:
(get_result('2019-01-14', '2019-01-17').groupby('name').count()==3)['start_d']

name
Поставщик 1     True
Поставщик 2     True
Поставщик 3    False
Name: start_d, dtype: bool